### Creating a basic LSTM Sequence-to-Sequence model for translating Japanese to English

In [13]:
import os
import json
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# get JSON file of sentence pairs
benchmark_directory = os.getcwd()
os.chdir(os.path.join(benchmark_directory, './..'))
sentences_file = open('./subtitle_corpus.json', 'r')
sentences_json = json.load(sentences_file)

In [3]:
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
# restrict to subsample before testing local runtimes
num_samples = 5000  # Number of samples to train on.
sentence_subset = sentences_json[:num_samples]

In [4]:
# handle empty data
def xstr(s):
    if s is None:
        return ''
    return str(s)

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for sentence in sentence_subset[: min(num_samples, len(sentence_subset) - 1)]:
    input_text, target_text = sentence['j'], sentence['e']
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + xstr(target_text) + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 4999
Number of unique input tokens: 2010
Number of unique output tokens: 52
Max sequence length for inputs: 82
Max sequence length for outputs: 173


In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [9]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [10]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [11]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [14]:
# Callback to save model at each epoch
callbacks = [
    ModelCheckpoint(
        filepath='s2s_{epoch}.h5',
        save_freq='epoch')
]

In [15]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=callbacks,
          validation_split=0.2)

Epoch 1/50
63/63 [==============================] - 54s 864ms/step - loss: 0.9838 - accuracy: 0.8088 - val_loss: 0.7215 - val_accuracy: 0.8195
Epoch 2/50
63/63 [==============================] - 54s 857ms/step - loss: 0.6821 - accuracy: 0.8232 - val_loss: 0.6576 - val_accuracy: 0.8222
Epoch 3/50
63/63 [==============================] - 54s 858ms/step - loss: 0.6693 - accuracy: 0.8259 - val_loss: 0.6294 - val_accuracy: 0.8267
Epoch 4/50
63/63 [==============================] - 53s 837ms/step - loss: 0.6079 - accuracy: 0.8353 - val_loss: 0.5762 - val_accuracy: 0.8382
Epoch 5/50
63/63 [==============================] - 52s 830ms/step - loss: 0.5666 - accuracy: 0.8456 - val_loss: 0.5434 - val_accuracy: 0.8441
Epoch 6/50
63/63 [==============================] - 53s 838ms/step - loss: 0.5234 - accuracy: 0.8522 - val_loss: 0.5168 - val_accuracy: 0.8541
Epoch 7/50
63/63 [==============================] - 53s 834ms/step - loss: 0.4998 - accuracy: 0.8571 - val_loss: 0.4994 - val_accuracy: 0.8571

In [16]:
# Save model
model.save('s2s.h5')

In [17]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [19]:
for seq_index in range(15):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: あなたは戻ったのね ハロルド?
Decoded sentence: i was a creating from your home.

-
Input sentence: 俺の相手は シャークだ。
Decoded sentence: i was a creating from the genst of the strangers to take any of the start.

-
Input sentence: 引き換えだ ある事とある物の
Decoded sentence: i was a creating from the genst of the strangers to take any of the start.

-
Input sentence: もういいよ ごちそうさま ううん
Decoded sentence: i was a creating from your him.

-
Input sentence: もう会社には来ないでくれ 電話もするな
Decoded sentence: i was a creating from your him.

-
Input sentence: きれいだ。
Decoded sentence: i was a creating from your him.

-
Input sentence: 連れて行け 殺しそうだ わかったか?
Decoded sentence: i was a creating from your him.

-
Input sentence: 殺したのか!
Decoded sentence: i was a creating from your him.

-
Input sentence: わぁ~! いつも すみません。 いいのよ~。
Decoded sentence: i was a creating from your him.

-
Input sentence: カンパニーの元社員が
Decoded sentence: i was a creating from the genst of the strangers to take any of the start.

-
Input sentence: じゃアイツラはどこ?
Deco